# Introduction

In this jupyter notebook, a CNN will be trained to classify Chest X-ray (CXR) images, whether they are diagnosed with COVID-19 or have diagnosis (NO FINDING). For this purpose, two seperate datasets are used. One that contains COVID-19 CXR images and another containing CXR images with no diagnosis (see below). Furthermore, a segmentation is applied with a U-Net to mask out parts of images irrelevant to a COVID-19 diagnosis (see below).

# Preparation of tools

## Import of required packages

In [ ]:
# Install required packages
!pip install tensorflow_addons

import datetime
import json
import os
import re
import tarfile
from pathlib import Path

import cv2
import dill
import numpy as np
import pandas as pd
import tensorflow as tf
from google.colab import drive
from matplotlib import pyplot as plt
from PIL import Image
from skimage import color, exposure, io, morphology, transform
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from tensorflow.keras import Model
from tensorflow.keras.applications import NASNetLarge
from tensorflow.keras.callbacks import (EarlyStopping, ModelCheckpoint,
                                        TensorBoard)
from tensorflow.keras.initializers import Constant
from tensorflow.keras.layers import (Activation, Dense, Dropout,
                                     GlobalMaxPooling2D, Input, LeakyReLU)
from tensorflow.keras.metrics import (AUC, BinaryAccuracy,
                                      CategoricalCrossentropy, FalseNegatives,
                                      FalsePositives,
                                      MeanAbsolutePercentageError, Precision,
                                      Recall, SensitivityAtSpecificity,
                                      SpecificityAtSensitivity, TrueNegatives,
                                      TruePositives)
from tensorflow.keras.models import Model, load_model, save_model
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow_addons.metrics import F1Score

[ChestX-ray8.tar](https://drive.google.com/file/d/1jqaMwk-VVvr7snrqcsGL_gj7TXp6-oWB/view?usp=sharing) must be added to your Google Drive as a shortcut. You possibly have to modify the path in line four on the cell below depending on where you have placed the shortcut of the `tar` file.

In [ ]:
drive_path = ('drive')
drive.mount(os.path.join(os.getcwd(), drive_path))

tar = tarfile.open('/content/drive/My Drive/Colab Notebooks/datasets/ChestX-ray8.tar', 'r')
tar.extractall(os.path.join(os.getcwd(), 'data'))

[](https://drive.google.com/file/d/1jqaMwk-VVvr7snrqcsGL_gj7TXp6-oWB/view?usp=sharing)

## Cloning necessary repositories 
- [covid-19-chestxray-dataset](https://github.com/ieee8023/covid-chestxray-dataset) into `data/covid-chestxray-dataset` is used for Chest X-rays with COVID-19 Diagnosis
- [lung-segmentation-2d](https://github.com/imlab-uiip/lung-segmentation-2d) into `tools/lung-segmentation-2d` is used to apply lung segmentation to the Chest X-ray images and create lung masks.

In [ ]:
!git clone https://github.com/ieee8023/covid-chestxray-dataset.git data/covid-chestxray-dataset
!git clone https://github.com/imlab-uiip/lung-segmentation-2d.git tools/lung-segmentation-2d

The configuration is defined in the following cell as a `dict`.

In [ ]:
drive_path = ('drive')
config = {
  'PATHS': {
    'RAW_DATA': os.path.join(os.getcwd(), 'data'),
    'COVID_CHEST_XRAY_DATA': os.path.join(os.getcwd(), 'data', 'covid-chestxray-dataset'),
    'CHEST_XRAY_8_DATA': os.path.join(os.getcwd(), 'data', 'ChestX-ray8'),
    'PROCESSED_DATA': os.path.join(os.getcwd(), 'data', 'processed'),
    'TRAIN_SET': os.path.join(os.getcwd(), 'data', 'processed', 'train_set.csv'),
    'VAL_SET': os.path.join(os.getcwd(), 'data', 'processed', 'val_set.csv'),
    'TEST_SET': os.path.join(os.getcwd(), 'data', 'processed', 'test_set.csv'),
    'IMAGES': os.path.join(os.getcwd(), drive_path, 'My Drive', 'Colab Notebooks', 'output', 'documents', 'generated_images'),
    'LOGS': os.path.join(os.getcwd(), 'results', 'logs'),
    'MODELS_FOLDER': os.path.join(os.getcwd(), drive_path, 'My Drive', 'Colab Notebooks', 'models', 'model_covid'),
    'OUTPUT_CLASS_INDICES': os.path.join(os.getcwd(), 'data', 'interpretability', 'output_class_indices.pkl'),
    'UNET_MODEL_PATH': os.path.join(os.getcwd(), 'tools', 'lung-segmentation-2d', 'trained_model.hdf5')
  },
  'DATA': {
    'IMG_DIM': tuple([331])*2,
    'VIEW': 'PA',
    'VAL_SPLIT_PERCENT': 0.08,
    'TEST_SPLIT_PERCENT': 0.1,
    'NUM_CHEST_XRAY_8_IMAGES': 1000,
    'RESIZE_WITH_PADDING': True,
    'CLASSES': [
      'NO FINDING',
      'COVID-19'
    ],
    'OTHER_CONTAINS_ONLY_HEALTHY': True,
    # One of 'class_weight' or 'reduce'
    'CLASS_BALANCE_STRATEGY': ['class_weight', 'reduce'][1]
  },
  'SEGMENTATION': {
    'IMG_DIM': tuple([256])*2,
    'MORPHOLOGY_KERNEL_SIZE': tuple([5])*2,
    'DILATION_KERNEL_SIZE': tuple([2])*2,
    'DILATION_ITERATIONS': 3,
    'MASK_BINARIZATION_TRESHOLD': 0.25
  },
  'TRAIN': {
    'BATCH_SIZE': 32,
    'FT_BATCH_SIZE': 16,
    'EPOCHS': 150,
    'USE_MASKED_IMAGES': True,
    'THRESHOLDS': 0.5,
    'ENABLE_EARLY_STOPPING': True,
    'PATIENCE_FOR_EARLY_STOPPING': 10,
    'NUM_GPUS': 1
  },
  'NN': {
    'NODES_DENSE0': 256,
    'LR': 0.00001,
    'FT_LR': 0.000001,
    'OPTIMIZER': 'adam',
    'DROPOUT': 0.3,
    'L2_LAMBDA': 0.0001
  },
  'PREDICTION': {
    'THRESHOLD': 0.5
  }
}

In [ ]:
# Create folders if not yet created
for path in config['PATHS']:
  if not bool(re.match('^.*\.[a-zA-Z0-9]+$', config['PATHS'][path])):
    Path(config['PATHS'][path]).mkdir(parents=True, exist_ok=True)
  else:
    splitted_path = config['PATHS'][path][:config['PATHS'][path].rfind('/')]
    Path(splitted_path).mkdir(parents=True, exist_ok=True)

# Data Preprocessing

In [ ]:
# process datasets in pandas for futher preprocessing
covid_chest_xray_path = config['PATHS']['COVID_CHEST_XRAY_DATA']
chest_xray_8_path = config['PATHS']['CHEST_XRAY_8_DATA']

covid_chest_xray_df = pd.read_csv(os.path.join(covid_chest_xray_path, 'metadata.csv'))
covid_chest_xray_df['filename'] = [os.path.join(covid_chest_xray_path, 'images', row) for row in covid_chest_xray_df['filename'].astype(str)]

covid_views_cxrs_df = covid_chest_xray_df['view'].str.match(config['DATA']['VIEW'])
covid_pos_df = covid_chest_xray_df['finding'].str.contains('COVID-19')
covid_df = covid_chest_xray_df[covid_pos_df & covid_views_cxrs_df] 

chest_xray_8_df = pd.read_csv(os.path.join(chest_xray_8_path, 'subset.csv'))
num_chest_xray_8_imgs = config['DATA']['NUM_CHEST_XRAY_8_IMAGES']
chest_xray_8_normal_df = chest_xray_8_df[chest_xray_8_df['Finding Labels'].str.match('No Finding')]
chest_xray_8_pneum_df = chest_xray_8_df[chest_xray_8_df['Finding Labels'].str.match('(?!No Finding)')]

chest_xray_8_normal_sample_df = chest_xray_8_normal_df.sample(frac = num_chest_xray_8_imgs / chest_xray_8_normal_df.shape[0], random_state=num_chest_xray_8_imgs)

chest_xray_8_pneum_sample_df = chest_xray_8_pneum_df.sample(frac = num_chest_xray_8_imgs / chest_xray_8_pneum_df.shape[0], random_state=num_chest_xray_8_imgs)

if config['DATA']['OTHER_CONTAINS_ONLY_HEALTHY']:
  chest_xray_8_df = chest_xray_8_normal_sample_df
else:
  chest_xray_8_df = pd.concat([chest_xray_8_normal_sample_df, chest_xray_8_pneum_sample_df], axis=0)

chest_xray_8_df['filename'] = [os.path.join(chest_xray_8_path, row) for row in chest_xray_8_df['Image Index'].astype(str)]

In [ ]:
# define labels and concatinate datasets
covid_df = covid_df.assign(label='COVID-19')
chest_xray_8_df = chest_xray_8_df.assign(label='NO FINDING')

chest_xray_8_selected_df = None
if config['DATA']['CLASS_BALANCE_STRATEGY'] == 'reduce':
  chest_xray_8_selected_df = chest_xray_8_df.head(covid_df.shape[0])
else:
  chest_xray_8_selected_df = chest_xray_8_df

file_df = pd.concat(
        [covid_df[['filename', 'label']],
        chest_xray_8_selected_df[['filename', 'label']]], axis=0)

## Padding

A padding is added to the images in order to bring them to an `n*n` dimension without distortion.

In [ ]:
def resize_with_pad(file_path, img_size):
  img = Image.open(file_path)
  old_size = img.size  
  ratio = float(img_size) / max(old_size)
  new_size = tuple([int(x * ratio) for x in old_size])

  img = img.resize(new_size, Image.ANTIALIAS)

  new_img = Image.new("RGB", (img_size, img_size))
  new_img.paste(img, ((img_size - new_size[0]) // 2,
                  (img_size - new_size[1]) // 2))
  return new_img

for file_path in file_df['filename']:
  if config['DATA']['RESIZE_WITH_PADDING']:
    padded_img = resize_with_pad(file_path, config['DATA']['IMG_DIM'][0])
    padded_img.save(file_path)

## Segmentation

Mask the lungs with the help of the U-Net

In [ ]:
UNet = load_model(config['PATHS']['UNET_MODEL_PATH'])

def mask_image(image, mask):
    """Returns masked image"""
    return np.ma.masked_where(mask == 0, image)
     

def remove_small_regions_and_dilate(image):
    """Morphologically removes small (less than kernel size) connected regions of 0s or 1s and dilates mask"""

    morphology_kernel = np.ones(config['SEGMENTATION']['MORPHOLOGY_KERNEL_SIZE'], np.uint8)
    dilation_kernel = np.ones(config['SEGMENTATION']['DILATION_KERNEL_SIZE'], np.uint8)
    image = np.squeeze(image).astype(np.float32)
    image = cv2.morphologyEx(image, cv2.MORPH_CLOSE, morphology_kernel)
    image = cv2.morphologyEx(image, cv2.MORPH_OPEN, morphology_kernel)
    image = cv2.dilate(image, dilation_kernel, iterations = config['SEGMENTATION']['DILATION_ITERATIONS'])

    return image

In [ ]:
masked_filenames = []

for row in file_df.iterrows():
  file_name = row[1][0]
  dot_index = file_name.rfind('.')
  masked_image_filename = '{}{}{}'.format(file_name[:dot_index], '_masked', file_name[dot_index:])
  mask_filename = '{}{}{}'.format(file_name[:dot_index], '_mask', file_name[dot_index:])
  original_image = cv2.imread(file_name, 0).astype(np.float32)/255.0
  original_image_size = original_image.shape[::-1]
  
  downsized_image = cv2.resize(original_image, dsize=config['SEGMENTATION']['IMG_DIM'], interpolation=cv2.INTER_CUBIC)
  downsized_image = np.expand_dims(downsized_image, axis=0)
  downsized_image = tf.image.per_image_standardization(downsized_image)

  mask_prediction = UNet.predict(downsized_image)

  mask = mask_prediction > config['SEGMENTATION']['MASK_BINARIZATION_TRESHOLD']
  mask = remove_small_regions_and_dilate(mask)
  upsized_mask = cv2.resize(np.squeeze(mask).astype(np.float32), dsize=original_image_size, interpolation=cv2.INTER_CUBIC)
  masked_image = mask_image(original_image, upsized_mask)
  cv2.imwrite(mask_filename, upsized_mask*255)
  cv2.imwrite(masked_image_filename, masked_image*255)
  masked_filenames.append(masked_image_filename)
  
file_df.insert(1, 'masked_filename', masked_filenames)

## Dataset split

Create Train, Test and Validation datasets

In [ ]:
validation_split_size = config['DATA']['VAL_SPLIT_PERCENT']
test_split_size = config['DATA']['TEST_SPLIT_PERCENT']
file_df_train, file_df_test = train_test_split(file_df, test_size=test_split_size, stratify=file_df['label'], random_state=42)
relative_validation_split_size = validation_split_size / (1 - test_split_size)
file_df_train, file_df_val = train_test_split(file_df_train, test_size=relative_validation_split_size,
                                                    stratify=file_df_train['label'], random_state=42)

if not os.path.exists(config['PATHS']['PROCESSED_DATA']):
    os.makedirs(config['PATHS']['PROCESSED_DATA'])
file_df_train.to_csv(config['PATHS']['TRAIN_SET'])
file_df_val.to_csv(config['PATHS']['VAL_SET'])
file_df_test.to_csv(config['PATHS']['TEST_SET'])

data = {}
data['TRAIN'] = pd.read_csv(config['PATHS']['TRAIN_SET'])
data['VAL'] = pd.read_csv(config['PATHS']['VAL_SET'])
data['TEST'] = pd.read_csv(config['PATHS']['TEST_SET'])

Remove images selected for the demonstrator from training dataset and replace them with images from the test dataset.

In [ ]:
demonstrator_image_list = ["00008720_000.png", '23E99E2E-447C-46E5-8EB2-D35D12473C39.png', '14d81f378173b86cc53f21d2d67040_jumbo.jpeg', '6C94A287-C059-46A0-8600-AFB95F4727B7.jpeg', '16660_4_1.jpg', '16660_5_1.jpg'] 
is_in_train = [train_image for train_image in list(data['TRAIN']['filename']) for demonstrator_image in demonstrator_image_list if demonstrator_image in str(train_image)]
is_in_train
is_in_val = [val_image for val_image in list(data['VAL']['filename']) for demonstrator_image in demonstrator_image_list if demonstrator_image in str(val_image)]
print(is_in_train)
print(is_in_val)

for file_path in is_in_train:
  data['TEST'] = data['TEST'].append(data['TRAIN'].loc[data['TRAIN']['filename'] == file_path])
  data['TRAIN'] = data['TRAIN'].loc[data['TRAIN']['filename'] != file_path]
  data['TRAIN'] = data['TRAIN'].append(data['TEST'].iloc[0])
  data['TEST'] = data['TEST'].tail(-1)

for file_path in is_in_val:
  data['TEST'] = data['TEST'].append(data['VAL'].loc[data['VAL']['filename'] == file_path])
  data['VAL'] = data['VAL'].loc[data['VAL']['filename'] != file_path]
  data['VAL'] = data['VAL'].append(data['TEST'].iloc[0])
  data['TEST'] = data['TEST'].tail(-1)

demonstrator_image_list = ["00008720_000.png", '23E99E2E-447C-46E5-8EB2-D35D12473C39.png', '14d81f378173b86cc53f21d2d67040_jumbo.jpeg', '6C94A287-C059-46A0-8600-AFB95F4727B7.jpeg', '16660_4_1.jpg', '16660_5_1.jpg'] 
is_in_train = [train_image for train_image in list(data['TRAIN']['filename']) for demonstrator_image in demonstrator_image_list if demonstrator_image in str(train_image)]
is_in_train
is_in_test = [val_image for val_image in list(data['TEST']['filename']) for demonstrator_image in demonstrator_image_list if demonstrator_image in str(val_image)]
print(len(is_in_test) == len(demonstrator_image_list))

Save datasets to csv files.

In [ ]:
if not os.path.exists(config['PATHS']['PROCESSED_DATA']):
    os.makedirs(config['PATHS']['PROCESSED_DATA'])
file_df_train.to_csv(config['PATHS']['TRAIN_SET'])
file_df_val.to_csv(config['PATHS']['VAL_SET'])
file_df_test.to_csv(config['PATHS']['TEST_SET'])

Here we define the callbacks for the training. They prevent overfitting and generally spare time when the further training of model does not improve performance.

# Training

In [ ]:
cur_date = datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
log_dir = os.path.join(config['PATHS']['LOGS'], 'training', cur_date)
model_filename_single = 'weights.hdf5'
if not os.path.exists(os.path.join(config['PATHS']['LOGS'], 'training')):
    os.makedirs(os.path.join(config['PATHS']['LOGS'], 'training'))
callbacks = []
if config['TRAIN']['ENABLE_EARLY_STOPPING']:
  early_stopping = EarlyStopping(
    monitor='val_auc_pr',
    verbose=1, 
    patience=config['TRAIN']['PATIENCE_FOR_EARLY_STOPPING'], 
    mode='max', 
    restore_best_weights=True)
  callbacks.append(early_stopping)
tensorboard = TensorBoard(log_dir=log_dir, histogram_freq=1)
callbacks.append(tensorboard)
model_checkpoint = ModelCheckpoint(model_filename_single, 
                                   monitor='val_auc_pr', 
                                   verbose=1, 
                                   save_best_only=True,
                                   save_weights_only=True, 
                                   mode='max', 
                                   save_freq='epoch'
)
callbacks.append(model_checkpoint)

Create `ImageDataGenerators` for the training, validation, and test datasets. Hereby the training dataset is augmented with random rotations of 10 degrees in both directions.

In [ ]:
train_img_gen = ImageDataGenerator(rotation_range=10, samplewise_std_normalization=True, samplewise_center=True)
val_img_gen = ImageDataGenerator(samplewise_std_normalization=True, samplewise_center=True)
test_img_gen = ImageDataGenerator(samplewise_std_normalization=True, samplewise_center=True)

img_shape = config['DATA']['IMG_DIM']

file_column = 'masked_filename' if config['TRAIN']['USE_MASKED_IMAGES'] else 'filename'

class_mode = 'binary'
train_generator = train_img_gen.flow_from_dataframe(
    dataframe=data['TRAIN'],
    x_col=file_column,
    y_col='label',
    target_size=img_shape,
    batch_size=config['TRAIN']['BATCH_SIZE'],
    class_mode=class_mode,
    validate_filenames=True)
val_generator = val_img_gen.flow_from_dataframe(
    dataframe=data['VAL'],
    x_col=file_column,
    y_col='label',
    target_size=img_shape,
    batch_size=config['TRAIN']['BATCH_SIZE'],
    class_mode=class_mode,
    validate_filenames=True)
test_generator = test_img_gen.flow_from_dataframe(
    dataframe=data['TEST'],
    x_col=file_column,
    y_col='label',
    target_size=img_shape,
    batch_size=config['TRAIN']['BATCH_SIZE'],
    class_mode=class_mode,
    validate_filenames=True,
    shuffle=False)

dill.dump(test_generator.class_indices, open(config['PATHS']['OUTPUT_CLASS_INDICES'], 'wb+'))

histogram = np.bincount(np.array(train_generator.labels).astype(int))

class_weight = None
if config['DATA']['CLASS_BALANCE_STRATEGY'] == 'class_weight':
  class_multiplier_list = [min(histogram) / max(histogram)]
  class_multiplier_list.insert(int(histogram[0] > histogram[1]), 1.0)

  class_multiplier = [
          class_multiplier_list[config['DATA']['CLASSES'].index(c)]
              for c in test_generator.class_indices
  ]

  weights = [(1.0 / len(histogram)) * sum(histogram) / histogram[i] for i in range(len(histogram))]

  class_weight = {i: class_multiplier[i] for i in range(len(histogram))}  

x,y = train_generator.next()
train_generator.reset()

fig, ax = plt.subplots(2,4)
fig.set_size_inches(15,15)
num = 0
for i in range(2):
  for j in range(4):
    ax[i,j].imshow(x[num])
    ax[i,j].axis('off')
    num += 1
        
plt.tight_layout()

## Defining the model and the training metrics

In [ ]:
covid_class_idx = test_generator.class_indices['COVID-19']   
thresholds = 1.0 / len(config['DATA']['CLASSES'])
metrics = [BinaryAccuracy(name='binary_accuracy'),
    Precision(name='precision', thresholds=thresholds, class_id=covid_class_idx),
    Recall(name='recall', thresholds=thresholds, class_id=covid_class_idx),
    AUC(name='auc_pr', curve='PR'),
    AUC(name='auc_roc', curve='ROC'),
    F1Score(name='f1score', threshold=thresholds, num_classes=1), 
    TrueNegatives(name='tn'), 
    TruePositives(name='tp'), 
    FalseNegatives(name='fn'), 
    FalsePositives(name='fp'),
    SpecificityAtSensitivity(sensitivity=thresholds, name='speAtSen'),
    SensitivityAtSpecificity(specificity=thresholds, name='senAtSpe')]

input_shape = config['DATA']['IMG_DIM']+tuple([3])
num_gpus = config['TRAIN']['NUM_GPUS']

model_config = config['NN']

nodes_dense0 = model_config['NODES_DENSE0']
lr = model_config['LR']
dropout = model_config['DROPOUT']
l2_lambda = model_config['L2_LAMBDA']

if model_config['OPTIMIZER'] == 'sgd':
    optimizer = SGD(learning_rate=lr)
else:
    optimizer = Adam(learning_rate=lr)

histogram = np.bincount([config['DATA']['CLASSES'].index(label) for label in data['TRAIN']['label'].astype(str)])
output_bias = np.log([histogram[i] / (np.sum(histogram) - histogram[i]) for i in range(histogram.shape[0])])

# Set output bias
if output_bias is not None:
    output_bias = Constant(output_bias)
print("MODEL CONFIG: ", model_config)

X_input = Input(input_shape, name='input_img')
base_model = NASNetLarge(include_top=False, weights='imagenet', input_shape=input_shape, input_tensor=X_input)
base_model.trainable = False
X = base_model(X_input, training=False)

# Add custom top
X = GlobalMaxPooling2D()(X)
X = Dropout(dropout)(X)
X = Dense(nodes_dense0, kernel_initializer='he_uniform', activity_regularizer=l2(l2_lambda))(X)
X = LeakyReLU()(X)
X = Dense(1)(X)
Y = Activation('sigmoid', dtype='float32', name='output')(X)

model = Model(inputs=X_input, outputs=Y)
model.summary()

model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=metrics)

## Training

In [ ]:
# Train model and save the training history
history = model.fit(train_generator, 
                    epochs=config['TRAIN']['EPOCHS'],        
                    validation_data=val_generator,
                    callbacks=callbacks,
                    class_weight=class_weight,
                    verbose=True)

## Evaluation

In [ ]:
# save history to json:
hist_df = pd.DataFrame(history.history) 
hist_json_file = '/content/history.json'
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

In [ ]:
#  evaluate model and save evaluation result to json:  
evaluation = model.evaluate(test_generator)
evaluation_data_df = pd.DataFrame(evaluation) 

evaluation_json_file = '/content/evaluation.json'
with open(evaluation_json_file, mode='w') as f:
    evaluation_data_df.to_json(f)

Save model to given Google Drive path with current data in the `config` dict and print file path.

In [ ]:
model_path = os.path.join(config['PATHS']['MODELS_FOLDER'], '{}{}{}'.format('model', cur_date, '.h5'))
save_model(model, model_path)
model_path

Save predictions.

In [ ]:
test_predictions = model.predict(test_generator)
test_labels = test_generator.labels
train_predictions = model.predict(train_generator)
train_labels = train_generator.labels
train = pd.DataFrame(zip([i[0] for i in train_predictions], train_labels), columns =['Predictions', 'Labels'])
test = pd.DataFrame(zip([i[0] for i in test_predictions], test_labels), columns =['Predictions', 'Labels'])
train.to_csv('train_predictions.csv')
test.to_csv('test_predictions.csv')

## Fine-tuning

Initalize fine tuning callbacks.

In [ ]:
cur_date = datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
log_dir = os.path.join(config['PATHS']['LOGS'], 'training_ft', cur_date)
model_filename_ft = 'fine_tuning_weights.hdf5'

if not os.path.exists(log_dir):
    os.makedirs(log_dir)

callbacks_ft = []
early_stopping_ft = EarlyStopping(
  monitor='val_auc_pr',
  verbose=1, 
  patience=3, 
  mode='max', 
  restore_best_weights=True)

callbacks_ft.append(early_stopping_ft)

tensorboard_ft = TensorBoard(log_dir=log_dir, histogram_freq=1)
callbacks_ft.append(tensorboard_ft)
model_checkpoint_ft = ModelCheckpoint(model_filename_ft, 
                                   monitor='val_auc_pr', 
                                   verbose=1, 
                                   save_best_only=True,
                                   save_weights_only=True, 
                                   mode='max', 
                                   save_freq='epoch', 
)
callbacks_ft.append(model_checkpoint_ft)

In [ ]:
file_column = 'masked_filename' if config['TRAIN']['USE_MASKED_IMAGES'] else 'filename'

class_mode = 'binary'
train_generator = train_img_gen.flow_from_dataframe(
    dataframe=data['TRAIN'],
    x_col=file_column,
    y_col='label',
    target_size=img_shape,
    batch_size=config['TRAIN']['FT_BATCH_SIZE'],
    class_mode=class_mode,
    validate_filenames=True)
val_generator = val_img_gen.flow_from_dataframe(
    dataframe=data['VAL'],
    x_col=file_column,
    y_col='label',
    target_size=img_shape,
    batch_size=config['TRAIN']['FT_BATCH_SIZE'],
    class_mode=class_mode,
    validate_filenames=True)
test_generator = test_img_gen.flow_from_dataframe(
    dataframe=data['TEST'],
    x_col=file_column,
    y_col='label',
    target_size=img_shape,
    batch_size=config['TRAIN']['FT_BATCH_SIZE'],
    class_mode=class_mode,
    validate_filenames=True,
    shuffle=False)

In [ ]:
model.get_layer(index=1).trainable = True

Configure the model, so that not only the custom top is trained, but also other parameters of the model.

In [ ]:
#base_model.trainable = True

lr = model_config['FT_LR']

if model_config['OPTIMIZER'] == 'sgd':
    optimizer = SGD(learning_rate=lr)
else:
    optimizer = Adam(learning_rate=lr)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=metrics)

model.summary()

## Fine-tuning evaluation

Fine tune the model for maximum 30 epochs.

In [ ]:
fine_tune_epochs = 30
last_epoch = history.epoch[-1] - config['TRAIN']['PATIENCE_FOR_EARLY_STOPPING'] + 1
total_epochs = last_epoch + fine_tune_epochs

history_fine = model.fit(train_generator, 
                          epochs=total_epochs,                          
                          initial_epoch=last_epoch,
                          validation_data=val_generator, 
                          callbacks=callbacks_ft,
                          verbose=True,
                          class_weight=class_weight)

Evaluate fine tuned model and save results.

In [ ]:
# save fine-tuning history to json:  

hist_ft_df = pd.DataFrame(history_fine.history) 

hist_ft_json_file = '/content/history_ft.json'
with open(hist_ft_json_file, mode='w') as f:
    hist_ft_df.to_json(f)

In [ ]:
# save evaluation to json:  
evaluation = model.evaluate(test_generator)

evaluation_data_df = pd.DataFrame(evaluation) 

evaluation_json_file = '/content/evaluation_ft.json'
with open(evaluation_json_file, mode='w') as f:
    evaluation_data_df.to_json(f)

Save fine tuned model.

In [ ]:
model_path = os.path.join(config['PATHS']['MODELS_FOLDER'], '{}{}{}'.format('model_ft', cur_date, '.h5'))
save_model(model, model_path)
model_path

In [ ]:
test_predictions = model.predict(test_generator)
test_labels = test_generator.labels
train_predictions = model.predict(train_generator)
train_labels = train_generator.labels
train = pd.DataFrame(zip([i[0] for i in train_predictions], train_labels), columns =['Predictions', 'Labels'])
test = pd.DataFrame(zip([i[0] for i in test_predictions], test_labels), columns =['Predictions', 'Labels'])
train.to_csv('train_predictions_ft.csv')
test.to_csv('test_predictions_ft.csv')

Copy result to Google Drive.

In [ ]:
!mv evaluation.json evaluation_ft.json history.json history_ft.json train_predictions.csv test_predictions.csv train_predictions_ft.csv test_predictions_ft.csv results

In [ ]:
!cp -r results "/content/drive/My Drive/Colab Notebooks/models/model_covid/"